In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm

In [93]:
    # with open('data/LI-Small_Trans.csv', 'r') as file:
Cov = pd.read_csv("data/Base.csv", 
                  sep=',')
Cov["device_fraud_count"].max()


device_fraud_count
0    1000000
Name: count, dtype: int64

In [4]:
Cov.head()

,fraud_bool,income,name_email_similarity,prev_address_months_count,current_address_months_count,customer_age,days_since_request,intended_balcon_amount,payment_type,zip_count_4w,...,has_other_cards,proposed_credit_limit,foreign_request,source,session_length_in_minutes,device_os,keep_alive_session,device_distinct_emails_8w,device_fraud_count,month
0,0,0.3,0.986506,-1,25,40,0.006735,102.453711,AA,1059,...,0,1500.0,0,INTERNET,16.224843,linux,1,1,0,0
1,0,0.8,0.617426,-1,89,20,0.010095,-0.849551,AD,1658,...,0,1500.0,0,INTERNET,3.363854,other,1,1,0,0
2,0,0.8,0.996707,9,14,40,0.012316,-1.490386,AB,1095,...,0,200.0,0,INTERNET,22.730559,windows,0,1,0,0
3,0,0.6,0.475100,11,14,30,0.006991,-1.863101,AB,3483,...,0,200.0,0,INTERNET,15.215816,linux,1,1,0,0
4,0,0.9,0.842307,-1,29,40,5.742626,47.152498,AA,2339,...,0,200.0,0,INTERNET,3.743048,other,0,1,0,0


In [5]:
# this will almost definitely be s h i t but i wanna see if i can find the laundering transactions by just applying a basic DNN classification model to the data
# drop the first row


# split Cov into data and target
target = Cov["fraud_bool"]
data = Cov.drop(columns=["fraud_bool"])

In [11]:
# get all columns that are not numeric
non_numeric_columns = data.select_dtypes(exclude=[np.number]).columns

# print the number of non-numeric columns
print(len(non_numeric_columns))
print(non_numeric_columns)

# get all columns that are numeric
numeric_columns = data.select_dtypes(include=[np.number]).columns

# print the number of numeric columns
print(len(numeric_columns))
print(numeric_columns)


5
Index(['payment_type', 'employment_status', 'housing_status', 'source',
       'device_os'],
      dtype='object')
26
Index(['income', 'name_email_similarity', 'prev_address_months_count',
       'current_address_months_count', 'customer_age', 'days_since_request',
       'intended_balcon_amount', 'zip_count_4w', 'velocity_6h', 'velocity_24h',
       'velocity_4w', 'bank_branch_count_8w',
       'date_of_birth_distinct_emails_4w', 'credit_risk_score',
       'email_is_free', 'phone_home_valid', 'phone_mobile_valid',
       'bank_months_count', 'has_other_cards', 'proposed_credit_limit',
       'foreign_request', 'session_length_in_minutes', 'keep_alive_session',
       'device_distinct_emails_8w', 'device_fraud_count', 'month'],
      dtype='object')


In [44]:
from sklearn.preprocessing import StandardScaler

# Create a scaler
scaler = StandardScaler()

for col in numeric_columns:
    data[col] = data[col].astype('float32')
 
 
# Fit the scaler to the training data and transform both training and test data
data_scaled = scaler.fit_transform(data[numeric_columns])

# Replace the original columns with the scaled ones
data[numeric_columns] = data_scaled

In [45]:
from sklearn.preprocessing import OrdinalEncoder

# Create the encoder
encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

# Fit the encoder to the non-numeric columns
categorical_columns = non_numeric_columns

encoder.fit(data[categorical_columns])

# Transform both the training and test data
encoded_data = encoder.transform(data[categorical_columns])

In [46]:
# replace the original columns with the encoded ones
data[categorical_columns] = encoded_data

In [47]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, target , test_size=0.6)
X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test, test_size=0.5)

In [48]:
# now we can create a DNN model
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

# Create the input layer
inputs = Input(shape=(31,))
# Create the first hidden layer
x = Dense(50, activation='relu')(inputs)
# Create the second hidden layer
x = Dense(50, activation='relu')(x)
# Create the output layer
outputs = Dense(1, activation='sigmoid')(x)

# Create a model
model = Model(inputs=inputs, outputs=outputs)

In [73]:
SaS = keras.metrics.SpecificityAtSensitivity(0.05)
AUC = keras.metrics.AUC()

In [74]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[SaS])

In [75]:
# Fit the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5)

Epoch 1/5
12500/12500 [==============================] - 28s 2ms/step - loss: 0.0454 - specificity_at_sensitivity_2: 0.9994 - val_loss: 0.0488 - val_specificity_at_sensitivity_2: 0.9989
Epoch 2/5
12500/12500 [==============================] - 42s 3ms/step - loss: 0.0451 - specificity_at_sensitivity_2: 0.9994 - val_loss: 0.0484 - val_specificity_at_sensitivity_2: 0.9989
Epoch 3/5
12500/12500 [==============================] - 37s 3ms/step - loss: 0.0450 - specificity_at_sensitivity_2: 0.9994 - val_loss: 0.0488 - val_specificity_at_sensitivity_2: 0.9990
Epoch 4/5
12500/12500 [==============================] - 39s 3ms/step - loss: 0.0449 - specificity_at_sensitivity_2: 0.9995 - val_loss: 0.0484 - val_specificity_at_sensitivity_2: 0.9988
Epoch 5/5
12500/12500 [==============================] - 40s 3ms/step - loss: 0.0447 - specificity_at_sensitivity_2: 0.9995 - val_loss: 0.0486 - val_specificity_at_sensitivity_2: 0.9988


In [76]:
# Evaluate the model
model.evaluate(X_valid, y_valid)

9375/9375 [==============================] - 14s 2ms/step - loss: 0.0478 - specificity_at_sensitivity_2: 0.9992


[0.0478225015103817, 0.9991844296455383]

In [77]:
# confusion matrix
from sklearn.metrics import confusion_matrix

y_pred = model.predict(X_valid)
y_pred = (y_pred > 0.5)
cm = confusion_matrix(y_valid, y_pred)
print(cm) 
# output @ 5 epochs using AUC:
# [[296593     33]
#  [  3343     31]]

# output @ 5 epochs using SpecificityAtSensitivity(0.5) and AUC:
# [[296714      8]
#  [  3267     11]]

# output @ 5 epochs using SpecificityAtSensitivity(0.5):
# [[296719      3]
#  [  3270      8]]

# output @ 5 epochs using SpecificityAtSensitivity(0.2):
# [[296668     54]
#  [  3222     56]]

# Lower specificity, higher sensitivity? 
# 

9375/9375 [==============================] - 14s 1ms/step
[[296678     44]
 [  3224     54]]


In [87]:
Cov["device_fraud_count"].value_counts()

device_fraud_count
0    1000000
Name: count, dtype: int64